In [246]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [302]:
df = pd.read_csv('dataset_241219.csv')
len(df)

97308

In [303]:
df.columns

Index(['date', 'adgroup_id', 'rn', 'impressions', 'bought', 'cost', 'sold',
       'earned', 'cpa', 'rpc', 'profit', 'max_cpa', 'max_cpc',
       'campaign_budget_amount', 'change_date_time', 'event_type', 'old_value',
       'new_value', 'prior', 'change_rate', 'cat'],
      dtype='object')

In [304]:
df.event_type.value_counts()

event_type
NO_CHANGE                                36037
UPDATE_CAMPAIGN_BUDGET_AMOUNT            29293
CREATE_CAMPAIGN_CRITERION_AD_SCHEDULE    14425
UPDATE_CAMPAIGN                           9106
UPDATE_AD_GROUP_TARGET_CPA                8180
UPDATE_AD_GROUP_STATUS                     267
Name: count, dtype: int64

In [305]:
# Договорились пока скипать задание расписания
df = df.loc[df.event_type != 'CREATE_CAMPAIGN_CRITERION_AD_SCHEDULE'].reset_index(drop=True)

In [306]:
# Для 1 дня считаем пред значение стейта как "без изменений"
df.loc[df.rn == 1, 'old_value'] = 'NO_CHANGE'

In [307]:
# Здесь я убираю задвоенные даты по адгруппе согласно приоритету
df = df.sort_values(by=['adgroup_id', 'date', 'rn', 'prior'])
new_df = pd.DataFrame(columns=df.columns)
dadg = df[['date', 'adgroup_id']].drop_duplicates().reset_index(drop=True)
for date, adg in tqdm(zip(dadg['date'], dadg['adgroup_id'])):
    new_df = pd.concat([new_df, df.loc[(df.adgroup_id == adg) & (df.date == date)].reset_index(drop=True)[:1]])
print(len(new_df))
new_df = new_df.reset_index()

0it [00:00, ?it/s]C:\Users\croxw\AppData\Local\Temp\ipykernel_19736\2833645154.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, df.loc[(df.adgroup_id == adg) & (df.date == date)].reset_index(drop=True)[:1]])
76055it [05:20, 236.99it/s]

76055


In [308]:
new_df

,index,date,adgroup_id,rn,impressions,bought,cost,sold,earned,cpa,...,max_cpa,max_cpc,campaign_budget_amount,change_date_time,event_type,old_value,new_value,prior,change_rate,cat
0,0,2023-12-27,25358,1,914,49,3.0900,1,0.0764,0.063061,...,5.0,0.01,3.0,NaN,NO_CHANGE,NO_CHANGE,NaN,2,NaN,NaN
1,0,2023-12-28,25358,2,404,24,0.7100,3,1.5408,0.029583,...,5.0,0.01,3.0,2023-12-29 13:14:09,UPDATE_CAMPAIGN_BUDGET_AMOUNT,3,11,2,3.666667,UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP
2,0,2023-12-29,25358,3,4157,308,10.8400,59,22.0487,0.035195,...,5.0,0.01,11.0,NaN,UPDATE_CAMPAIGN_BUDGET_AMOUNT,3.0000,11.0000,2,3.666667,UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP
3,0,2023-12-30,25358,4,3950,338,10.7500,59,15.3252,0.031805,...,5.0,0.01,11.0,NaN,NO_CHANGE,NaN,NaN,2,NaN,NaN
4,0,2023-12-31,25358,5,3865,255,11.1800,50,8.1239,0.043843,...,5.0,0.01,11.0,NaN,NO_CHANGE,NaN,NaN,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76050,0,2024-12-18,138885,1,106,8,1.2135,1,0.1261,0.151688,...,0.0,0.00,1.0,NaN,NO_CHANGE,NO_CHANGE,NaN,2,NaN,NaN
76051,0,2024-12-18,138886,1,364,11,1.0657,1,0.0820,0.096882,...,0.0,0.00,1.0,NaN,NO_CHANGE,NO_CHANGE,NaN,2,NaN,NaN
76052,0,2024-12-18,138887,1,111,9,1.1086,2,0.0875,0.123178,...,0.0,0.00,1.0,NaN,NO_CHANGE,NO_CHANGE,NaN,2,NaN,NaN
76053,0,2024-12-18,138888,1,244,7,1.0294,1,0.1893,0.147057,...,0.0,0.00,1.0,NaN,NO_CHANGE,NO_CHANGE,NaN,2,NaN,NaN


In [310]:
# Здесь размечаю стейт стратегии группы (если в какой-то момент был изменена стратегия на TARGET_CPA, то последующие записи должны иметь соответствующий стейт стратегии)
max_conv = [1] * len(new_df)
for index, row in tqdm(new_df[['adgroup_id', 'new_value']].iterrows()):
    if row['new_value'] == 'TARGET_CPA':
        adg = row['adgroup_id']
        ind = index
        if len(new_df.loc[(new_df.new_value == 'MAXIMIZE_CONVERSIONS') & (new_df.index > ind) & (new_df.adgroup_id == adg)]) > 0:
            max_ind = min(max(new_df.loc[new_df.adgroup_id==adg].index), min(new_df.loc[(new_df.new_value == 'MAXIMIZE_CONVERSIONS') & (new_df.index > ind) & (new_df.adgroup_id == adg)].index))
        else:
            max_ind = max(new_df.loc[new_df.adgroup_id==adg].index)
        for i in range(ind, max_ind):
            max_conv[i] = 0
new_df['is_max_conversion'] = max_conv

76055it [00:17, 4359.95it/s]


In [311]:
# Разметка категории стратегии
new_df.loc[new_df.new_value == 'TARGET_CPA', 'cat'] = 'STRATEGY_CPA'
new_df.loc[new_df.new_value == 'MAXIMIZE_CONVERSIONS', 'cat'] = 'STRATEGY_CONVERSION'

In [ ]:
new_df.loc[new_df.new_value == 'TARGET_CPA', 'is_max_conversion'] = 1
new_df.loc[new_df.new_value == 'MAXIMIZE_CONVERSIONS', 'is_max_conversion'] = 0

In [385]:
df = new_df

In [386]:
df["event_lagged"] = df.event_type.shift(1)
df['target'] = df['cat'].combine_first(df['event_type'])

In [387]:
df.target.value_counts()

target
NO_CHANGE                             36037
UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP      22735
UPDATE_CAMPAIGN                        6007
STRATEGY_CPA                           2894
UPDATE_AD_GROUP_TARGET_CPA_UP          2858
UPDATE_CAMPAIGN_BUDGET_AMOUNT_DOWN     2738
UPDATE_AD_GROUP_TARGET_CPA_DOWN        2522
UPDATE_AD_GROUP_STATUS                  217
STRATEGY_CONVERSION                      40
UPDATE_CAMPAIGN_BUDGET_AMOUNT             7
Name: count, dtype: int64

In [388]:
ndf = df.loc[~( ((df.event_type == 'CREATE_AD_GROUP') | (df.event_type == 'CREATE_CAMPAIGN_CRITERION_AD_SCHEDULE') | (df.event_type == 'UPDATE_AD_GROUP_STATUS')) | ((df.event_type == 'UPDATE_AD_GROUP_STATUS') & (df.event_lagged == 'CREATE_AD_GROUP')) | (df.event_type == 'CREATE_AD_GROUP') )][['adgroup_id', 'date', 'impressions', 'bought', 'cost', 'sold', 'earned', 'profit', 'max_cpa', 'max_cpc', 'campaign_budget_amount', 'rn', 'is_max_conversion', 'target']].reset_index(drop=True)
ndf = ndf.loc[ndf.target != 'UPDATE_CAMPAIGN_BUDGET_AMOUNT'].reset_index(drop=True)

In [ ]:
ndf['cum_profit'] = ndf.groupby('adgroup_id')['profit'].cumsum()
ndf['campaign_budget_amount'] = ndf['campaign_budget_amount'].fillna(5.0)
ndf['cost_rate'] = ndf['cost'] / ndf['campaign_budget_amount']
shifted = ndf.groupby("adgroup_id")['profit'].shift(1)
shifted_impr = ndf.groupby("adgroup_id")['impressions'].shift(1)
shifted_tgt= ndf.groupby("adgroup_id")['target'].shift(1)
shifted_cpa = ndf.groupby("adgroup_id")['max_cpa'].shift(1)
shifted_budget= ndf.groupby("adgroup_id")['campaign_budget_amount'].shift(1)
#ndf['impr'] = np.log(ndf['impressions'])
ndf['profit_lag'] = shifted
ndf['impressions_lag'] = shifted_impr
ndf['tgt_lag'] = shifted_tgt
ndf['profit_dif'] = (ndf['profit'] - ndf['profit_lag']) / (ndf['cum_profit'])
ndf['profit_rate'] = ndf['profit'] / ndf['cum_profit']
ndf['impr_dynamic'] = (ndf['impressions'] - ndf['impressions_lag']) / 1e3
ndf['profit_dif'] = ndf['profit_dif'].fillna(0.15)
ndf['impr_dynamic'] = ndf['impr_dynamic'].fillna(0.01)
ndf['profit_cond'] = ndf['profit'].apply(lambda x: 1 if x > 15 else 0)

ndf = pd.get_dummies(ndf, columns=['tgt_lag'], dtype=int)

def mark_cpa(a,b,c):
    return 1 if a<0 and b>0 and c>5 else 0
def mark_impr(a,b,c):
    return 1 if a<10000 and b>5 and c>5 else 0

ndf['cpa_flag'] = ndf.apply(lambda x: mark_cpa(x['profit_dif'], x['impr_dynamic'], x['rn']), axis=1)
ndf['impr_flag'] = ndf.apply(lambda x: mark_impr(x['impressions'], x['profit'], x['rn']), axis=1)

In [ ]:
shifted_costrate = ndf.groupby("adgroup_id")['cost_rate'].shift(1)
ndf['cost_rate_lag'] = shifted_costrate
ndf['cost_rate_dynamic'] = ndf['cost_rate_lag'] - ndf['cost_rate']
ndf['cost_rate_dynamic'] = ndf['cost_rate_dynamic'].fillna(0.15)
ndf['cost_rate_lag'] = ndf['cost_rate_lag'].fillna(0)

ndf['max_cpa'] = ndf['max_cpa'].fillna(method='bfill')
ndf['max_cpc'] = ndf['max_cpc'].fillna(method='bfill')
ndf['profit_rate'] = ndf['profit_rate'].fillna(0)

In [391]:
ind = ndf.loc[ndf.rn.isin([1,2,3])].index
ndf['is_first_days'] = 0
ndf.loc[ndf.index.isin(ind), 'is_first_days'] = 1

In [425]:
X.columns

Index(['impressions', 'impr_dynamic', 'cpa_flag', 'impr_flag', 'bought',
       'cost', 'sold', 'earned', 'profit', 'profit_cond', 'cost_rate',
       'cost_rate_dynamic', 'profit_dif', 'profit_rate', 'cum_profit',
       'max_cpa', 'max_cpc', 'rn', 'is_first_days', 'campaign_budget_amount',
       'is_max_conversion', 'tgt_lag_NO_CHANGE',
       'tgt_lag_UPDATE_AD_GROUP_TARGET_CPA_DOWN',
       'tgt_lag_UPDATE_AD_GROUP_TARGET_CPA_UP',
       'tgt_lag_UPDATE_CAMPAIGN_BUDGET_AMOUNT_DOWN',
       'tgt_lag_UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP'],
      dtype='object')

               Коротенько про атрибуты
is_max_conversion - флаг текущей стратегии группы
target - действие, которое будет совершено по итогам текущего дня
cum_profit - куммулятивный профит с момента запуска группы
cost_rate - оценка затрат группы относительно размера бюджета кампании
cost_rate_dynamic - cost_rate в динамике
profit_dif - динамическая оценка профита в динамике
profit_rate - отношение профита к куммулятивному профиту
impr_dynamic - динамическая оценка impressions в динамике
is_first_days - является ли текущий день одним из первых трех дней группы
tgt_lag_NO_CHANGE и прочие подобные - значение таргета за предыдущий день

In [392]:
cl = dict(zip(list(ndf['target'].value_counts().keys()), range(len(ndf['target'].value_counts()))))
mapping = dict(zip(list(ndf['target'].value_counts().keys()), range(len(ndf['target'].value_counts()))))
mapping

{'NO_CHANGE': 0,
 'UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP': 1,
 'UPDATE_CAMPAIGN': 2,
 'STRATEGY_CPA': 3,
 'UPDATE_AD_GROUP_TARGET_CPA_UP': 4,
 'UPDATE_CAMPAIGN_BUDGET_AMOUNT_DOWN': 5,
 'UPDATE_AD_GROUP_TARGET_CPA_DOWN': 6,
 'STRATEGY_CONVERSION': 7}

In [ ]:
# По названию классов можно понять что они означают кроме UPDATE_CAMPAIGN. Фактически, это пауза группы

In [423]:
ndf.to_csv('dataset_final.csv', index=False)

In [136]:
# MODEL

In [331]:
class model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(model, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.sigmoid = nn.Sigmoid()
        self.l2 = nn.Linear(hidden_size, hidden_size * 3) 
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(hidden_size * 3, hidden_size * 5) 
        self.tan = nn.ReLU()
        self.drop = nn.Dropout(0.1)
        self.l4 = nn.Linear(hidden_size * 5, hidden_size * 3) 
        self.relu = nn.ReLU()
        self.l5 = nn.Linear(hidden_size * 3, num_classes)   

    def forward(self, x):
        out = self.l1(x)
        out = self.sigmoid(out)
        out = self.l2(out)
        out = self.tan(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.drop(out)
        out = self.l4(out)
        out = self.relu(out)
        out = self.l5(out)
        return out 

In [329]:
ndf.loc[(ndf.is_max_conversion==1) & ((ndf.target == 'UPDATE_AD_GROUP_TARGET_CPA_UP') | (ndf.target == 'UPDATE_AD_GROUP_TARGET_CPA_DOWN')), 'target'] = 'NO_CHANGE'

In [ ]:
new_df.loc[(new_df.target == 'TARGET_CPA')&(new_df.is_max_conversion==0, 'cat'] = 'STRATEGY_CPA'
new_df.loc[new_df.new_value == 'MAXIMIZE_CONVERSIONS', 'cat'] = 'STRATEGY_CONVERSION'

In [394]:
cl

{'NO_CHANGE': 0,
 'UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP': 1,
 'UPDATE_CAMPAIGN': 2,
 'STRATEGY_CPA': 3,
 'UPDATE_AD_GROUP_TARGET_CPA_UP': 4,
 'UPDATE_CAMPAIGN_BUDGET_AMOUNT_DOWN': 5,
 'UPDATE_AD_GROUP_TARGET_CPA_DOWN': 6,
 'STRATEGY_CONVERSION': 7}

In [138]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

my_model = model(26, 42, 8).to(device)
my_model.load_state_dict(torch.load('model/model.pth'))

In [395]:
features = ['impressions', 'impr_dynamic', 'cpa_flag', 'impr_flag', 'bought', 'cost', 'sold', 
            'earned', 'profit', 'profit_cond', 'cost_rate', 'cost_rate_dynamic', 'profit_dif', 'profit_rate', 
            'cum_profit', 'max_cpa', 'max_cpc', 'rn', 'is_first_days', 'campaign_budget_amount', 'is_max_conversion',
       'tgt_lag_NO_CHANGE', 
       'tgt_lag_UPDATE_AD_GROUP_TARGET_CPA_DOWN',
       'tgt_lag_UPDATE_AD_GROUP_TARGET_CPA_UP',
       'tgt_lag_UPDATE_CAMPAIGN_BUDGET_AMOUNT_DOWN',
       'tgt_lag_UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP', 'target']

In [396]:
X = ndf[features[:-1]].astype(float)
y = ndf['target'].apply(lambda x: cl[x])

In [397]:
def get_key(val, my_dict):
    for key, value in my_dict.items():
        if val == value:
            return key

In [402]:
md = model(26, 42, 8).to(device)

In [403]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 42, stratify=y)

In [339]:
CUDA_LAUNCH_BLOCKING=1

In [408]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(md.parameters(), lr=0.000005) 

n_total_steps = len(X_train)
for epoch in range(100):
    for i in range(1, 50):  
        md.train()
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        
        data = torch.tensor(X_train.values, dtype = torch.float32)
        data = data.to(device)

        labels = torch.tensor(y_train.values)
        labels = labels.to(device)
        # Forward pass
        outputs = md(data)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 

        if (epoch+1) % 10 == 0 and i == 49:
            print (f'Epoch [{epoch+1}/{100}], Step[{i+1}], Loss: {loss.item():.4f}') 

    with torch.no_grad():
        outs = md(torch.tensor(X_test.values, dtype = torch.float32).to(device))
        _, predicted = torch.max(outs, 1)
        pred = predicted.detach().cpu().numpy()
        if (epoch+1) % 10 == 0:
            print(f1_score(y_test, pred, average='weighted'))

Epoch [10/100], Step[50], Loss: 0.7406
0.6288764609452809
Epoch [20/100], Step[50], Loss: 0.7403
0.6308310430564662
Epoch [30/100], Step[50], Loss: 0.7395
0.6290610493964277
Epoch [40/100], Step[50], Loss: 0.7416
0.6301215931072094
Epoch [50/100], Step[50], Loss: 0.7387
0.6329324142909368
Epoch [60/100], Step[50], Loss: 0.7378
0.6320860640782072
Epoch [70/100], Step[50], Loss: 0.7387
0.630029449363874
Epoch [80/100], Step[50], Loss: 0.7382
0.631991172737522
Epoch [90/100], Step[50], Loss: 0.7387
0.6302044161982789
Epoch [100/100], Step[50], Loss: 0.7388
0.6302026699726888


In [409]:
import copy 

X_t = copy.deepcopy(X)
y_t = copy.deepcopy(y)

In [410]:
with torch.no_grad():
    outs = md(torch.tensor(X_t.values, dtype = torch.float32).to(device))
    _, predicted_all = torch.max(outs, 1)
    preds = predicted_all.detach().cpu().numpy()
    print(f1_score(y_t, preds, average='weighted'))

fact= [get_key(x, cl) for x in y_t]
lbl = [get_key(x, cl) for x in preds]

0.6589049412003891


In [411]:
X_t['pred'] = lbl
X_t['fact'] = fact

In [422]:
torch.save(md.state_dict(), 'model/model_25_12.pth')

In [413]:
X_t.loc[X_t.is_max_conversion==0]['pred'].value_counts()

pred
NO_CHANGE                             19518
UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP       8463
UPDATE_CAMPAIGN_BUDGET_AMOUNT_DOWN     1848
UPDATE_AD_GROUP_TARGET_CPA_UP          1821
UPDATE_CAMPAIGN                        1021
UPDATE_AD_GROUP_TARGET_CPA_DOWN         404
Name: count, dtype: int64

In [414]:
pred = predicted_all.detach().cpu().numpy()
print(classification_report(y_t, pred, target_names=cl.keys(), digits=4, zero_division=0))
print(f1_score(y_t, pred, average='weighted'))

                                    precision    recall  f1-score   support

                         NO_CHANGE     0.6850    0.7872    0.7325     36037
  UPDATE_CAMPAIGN_BUDGET_AMOUNT_UP     0.7158    0.7968    0.7541     22735
                   UPDATE_CAMPAIGN     0.5772    0.3541    0.4389      6007
                      STRATEGY_CPA     0.4858    0.1945    0.2778      2894
     UPDATE_AD_GROUP_TARGET_CPA_UP     0.5082    0.3272    0.3980      2858
UPDATE_CAMPAIGN_BUDGET_AMOUNT_DOWN     0.6759    0.4974    0.5731      2738
   UPDATE_AD_GROUP_TARGET_CPA_DOWN     0.3907    0.0630    0.1086      2522
               STRATEGY_CONVERSION     0.0000    0.0000    0.0000        40

                          accuracy                         0.6809     75831
                         macro avg     0.5048    0.3775    0.4104     75831
                      weighted avg     0.6609    0.6809    0.6589     75831

0.6589049412003891


In [421]:
acc = 0
is_two_sided = 0
for index, row in X_t.iterrows():
    pr, fct = row['pred'], row['fact']
    if is_two_sided:
        if pr in list(X_t['fact'][index-2:index+2]):
            acc += 1
    else:
        if pr in list(X_t['fact'][index:index+2]):
            acc += 1
print(acc/len(X_t))

0.915786419801928
